In [3]:
from datetime import datetime

import biosppy as bp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

from ltbio.biosignals import modalities, Event
from ltbio.biosignals.sources.BitalinoShort import BitalinoShort


In [4]:

path_ = 'D:\\PreEpiSeizures\\Patients_HEM\\FCSFDM\\Bitalino'
options = {'date1': datetime(2021, 4, 15, 14, 45, 49),
           'date2': datetime(2021, 4, 15, 14, 49, 32)}
data = BitalinoShort._read(dir=path_, type=modalities.RESP, **options)

In [5]:
sig = data['PZT_chest']

{'ACCX_chest': <ltbio.biosignals.timeseries.Timeseries.Timeseries at 0x1e205c9afb0>,
 'ACCY_chest': <ltbio.biosignals.timeseries.Timeseries.Timeseries at 0x1e205c9b430>,
 'ACCZ_chest': <ltbio.biosignals.timeseries.Timeseries.Timeseries at 0x1e2643bc190>}

In [ ]:
event1 = Event('crise 4', datetime(2021, 4, 15, 14, 47, 32))

In [ ]:
resp_info = bp.signals.resp.resp(sig.to_array())

In [ ]:
rrate_idx = (resp_info['resp_rate_ts'] * sig.sampling_frequency).astype(int)
rrate = 60 * resp_info['resp_rate']
resp_sig = resp_info['filtered']
ts = resp_info['ts']

In [ ]:
sb.set(font_scale=2)
sb.set_style(style='white')
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(25, 7))

# plt.title('Respiratory Rate During Seizure')
ax2.set_ylabel('Breaths per minute')
ax2.set_xlabel('Time (s)')
ax1.set_ylabel('Relative Amplitude')
ax1.set_xlabel('Time (s)')
ax2.set_title('Respiratory Rate During FAS')
ax1.set_title('Respiration Signal During FAS')

x_axis = pd.date_range(sig.initial_datetime, sig.final_datetime, periods=len(sig))
ax2.plot(x_axis[rrate_idx][x_axis[rrate_idx] <= event1.onset], rrate[:len(x_axis[rrate_idx][x_axis[rrate_idx] <= event1.onset])], linewidth=3,
         color='#A9D1ED', label='preictal')
ax1.plot(x_axis[x_axis <= event1.onset], resp_sig[:len(x_axis[x_axis <= event1.onset])], linewidth=3,
         color='#A9D1ED', label='preictal')

ax1.plot(x_axis[x_axis >= event1.onset], resp_sig[-len(x_axis[x_axis >= event1.onset]):], linewidth=2, color='#F8CBAD', label='ictal')
ax2.plot(x_axis[rrate_idx][x_axis[rrate_idx] >= event1.onset], rrate[-len(x_axis[rrate_idx][x_axis[rrate_idx] >= event1.onset]):], linewidth=2, color='#F8CBAD', label='ictal')


from matplotlib.dates import DateFormatter
ax1.vlines(event1.onset, np.max(resp_sig), np.min(resp_sig), color= '#843C0C', label='Seizure Onset', linewidth=3)
ax2.vlines(event1.onset, np.max(rrate), np.min(rrate), color= '#843C0C', label='Seizure Onset', linewidth=3)

ax1.xaxis.set_major_formatter(DateFormatter('%M:%S'))
ax2.xaxis.set_major_formatter(DateFormatter('%M:%S'))


ax1.legend()
ax2.legend()
plt.tight_layout()
figname = 'C:\\Users\\Mariana\\Documents\\Epilepsy\\images\\resp_during_seizure'
plt.savefig(figname + '.png')
plt.savefig(figname + '.eps', format='eps')
plt.show()
